In [ ]:
from math import sin

def loop_slow(num_iterations):
    result = 0
    for i in xrange(num_iterations):
        result += i * sin(num_iterations)
    return result

def loop_fast(num_iterations):
    result = 0 
    factor = sin(num_iterations)
    for i in range(num_iterations):
        result += i
    return result * factor

In [ ]:
%load_ext line_profiler

In [ ]:
# Pure Python 2D diffusion profiling
from diffusion_python import run_experiment
%lprun -f run_experiment run_experiment(500)

'''
Timer unit: 1e-06 s

Total time: 178.527 s
File: /home/luno/Workspace/high_performance_python/diffusion_python.py
Function: run_experiment at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    23                                           def run_experiment(num_iterations):
    24                                               # setting up initial conditions
    25         1            9      9.0      0.0      xmax, ymax = grid_shape
    26         1           16     16.0      0.0      grid = [[0.0, ] * ymax] * xmax
    27                                           
    28                                               # initialization assumes that xmax <= ymax
    29         1           14     14.0      0.0      block_low = int(xmax * .4)
    30         1            4      4.0      0.0      block_high = int(xmax * .5)
    31        53           86      1.6      0.0      for i in range(block_low, block_high):
    32      2756         4013      1.5      0.0          for j in range(block_low, block_high):
    33      2704         4066      1.5      0.0              grid[i][j] = 0.005
    34                                           
    35         1           15     15.0      0.0      start = time.time()
    36       501         1027      2.0      0.0      for i in range(num_iterations):
    37       500    178518229 357036.5    100.0          grid = evolve(grid, 0.1)
    38         1            3      3.0      0.0      return time.time() - start
'''

In [ ]:
# Pure Python 2D diffusion after reducing memory allocation
from diffusion_python_memory import run_experiment
%lprun -f run_experiment run_experiment(500)

'''
Timer unit: 1e-06 s

Total time: 171.316 s
File: /home/luno/Workspace/high_performance_python/diffusion_python_memory.py
Function: run_experiment at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    19                                           def run_experiment(num_iterations):
    20                                               # setting up initial conditions
    21         1            6      6.0      0.0      xmax, ymax = grid_shape
    22         1           10     10.0      0.0      grid = [[0.0, ] * ymax] * xmax
    23         1           17     17.0      0.0      scratch = [[0.0, ] * ymax] * xmax
    24                                           
    25                                               # initialization assumes that xmax <= ymax
    26         1           10     10.0      0.0      block_low = int(xmax * .4)
    27         1            3      3.0      0.0      block_high = int(xmax * .5)
    28        53           79      1.5      0.0      for i in range(block_low, block_high):
    29      2756         5834      2.1      0.0          for j in range(block_low, block_high):
    30      2704         4629      1.7      0.0              grid[i][j] = 0.005
    31                                           
    32         1            7      7.0      0.0      start = time.time()
    33       501          284      0.6      0.0      for i in range(num_iterations):
    34       500    171304020 342608.0    100.0          evolve(grid, 0.1, scratch)
    35       500         1254      2.5      0.0          grid, scratch = scratch, grid
    36         1            3      3.0      0.0      return time.time() - start
'''

In [ ]:
# Python lists store pointers: it stores data's location, not holding the data.
# So there must be multiple lookups when fetching an element from `grid` matrix.

# Memory fragmentation: The data located in one continuous block in memory, 
# we could move all of the data in one operation - this could occur memory fragmentation.
# memory transfer overhead: such as `cache-misses`

# https://hbfs.wordpress.com/2013/01/08/python-memory-management-part-ii/

# But how do we know what data will be needed in the future?
# => CPU's branch prediction and pipelining

# Probing how well memory is being moved to the CPU can be quite hard
# => Linux's `perf`

# sudo apt-get install linux-tools-generic
# sudo apt-get install linux-cloud-tools-generic

```
> perf stat -e cycles,stalled-cycles-frontend,stalled-cycles-backend,instructions,\
cache-references,cache-misses,branches,branch-misses,task-clock,faults,\
minor-faults,cs,migrations -r 3 python diffusion_python_memory.py

 Performance counter stats for 'python diffusion_python_memory.py' (3 runs):

   228,108,155,330      cycles                    #    3.574 GHz                      ( +-  1.07% )
   <not supported>      stalled-cycles-frontend
   <not supported>      stalled-cycles-backend
   663,731,493,774      instructions              #    2.91  insns per cycle          ( +-  0.00% )
       176,306,169      cache-references          #    2.762 M/sec                    ( +- 17.48% )
        93,035,988      cache-misses              #   52.770 % of all cache refs      ( +-  3.24% )
   130,599,416,533      branches                  # 2045.993 M/sec                    ( +-  0.00% )
       429,253,502      branch-misses             #    0.33% of all branches          ( +-  4.84% )
      63831.814032      task-clock (msec)         #    0.996 CPUs utilized            ( +-  1.28% )
               841      faults                    #    0.013 K/sec                    ( +-  0.04% )
               841      minor-faults              #    0.013 K/sec                    ( +-  0.04% )
             4,213      cs                        #    0.066 K/sec                    ( +- 66.20% )
                 1      migrations                #    0.000 K/sec                    ( +-100.00% )

      64.081283044 seconds time elapsed                                          ( +-  1.51% )
```

- `task-clock`: how many clock cycles our task took, the CPU utilization isn't exactly 1 because there were periods where the process relied on other subsystems to do instructions for it
- `context-switches`: how the program is halted in order to wait for a kernel operation to finish.`CPU-migrations`: when the program is halted and resumed on a different CPU 
- `page-fault`: the modern Unix memory allocation scheme. when the memory is first used, the operating system throws a minor page fault interrupt, which pause the program that is being run and properly allocates the memory. (lazy allocation system). minor page faults: outside of the scope of the program, major page fault: when the program requests data from a device (disk, network, etc.) that hasn't been read yet.
- `cache-reference`: whenever we reference the data that is in our cache, this increases. `cache-miss`: when we fetch data from RAM, counts as a `cache-miss` (CPU bound work)
- `intructions`: how many instructions our code is issuing to the CPU. `insns per cycle`: Because of pipelining, these instructions can be run several at a time. `stalled-cycles-frontend` and `stalled-cycles-backed`: how many cycles our program was waiting for the frontend or backend of the pipeline to be filled. frontend: responsible for fetching the next instruction from memory and decoding it into a valid operation, backend: actually running the operation. Pipelining: run the current operation while fetching and preparing the next one.
- `branch`: a time in the code where the execution flow changes (like if.. then). for optimizing this, the CPU tries to guess which direction the branch will take and preload the relevant instructions. we will get `stalled-cycles` and `branch-miss` when this prediction is in incorrect.
- http://stackoverflow.com/questions/22712956/why-does-perf-stat-show-stalled-cycles-backend-as-not-supported
- http://www.bnikolic.co.uk/blog/hpc-prof-events.html

- L1/L2 cache 35,497,196 times
- Of those of them, 10,716,972 (30.1%) were requests for data that wasn't in memory at the time
- And an average of 1.82 instructions (total speed boost from pipelining, out-of-order execution, and hyperthreading)
- Fragmentations prevent vectorizaion of computations (or having the CPU do multiple computations at a time) Since the bus can only move contiguous chunks of memory, this is only possible if the grid data is stored sequentially in RAM. But the actual values in Python lists in the grid are scattered througout memory and cannot be copied all at once.

- `array` module instead of lists, stores data sequentially in memory, so that a lice of the array actually represents a continuous range in memory.
- But Python dosen't know how to vectorize our loops (no bytecode optimization in Python)
- looping an `array` is slower than looping an `list`
- not for math, more suitable for storing fixed-type data more efficiently in memory.

In [1]:
from array import array
import numpy

def norm_square_list(vector):
    norm = 0
    for v in vector:
        norm += v*v
    return norm

def norm_square_list_comprehension(vector):
    return sum([v*v for v in vector])

def norm_square_generator_comprehension(vector):
    return sum([v*v for v in vector])

def norm_square_array(vector):
    norm = 0
    for v in vector:
        norm += v*v
    return norm

def norm_square_numpy(vector):
    return numpy.sum(vector * vector)

def norm_square_numpy_dot(vector):
    return numpy.dot(vector, vector)

In [2]:
vector = range(1000000)

In [3]:
%timeit norm_square_list(vector)

10 loops, best of 3: 63.1 ms per loop


In [4]:
%timeit norm_square_list_comprehension(vector)

10 loops, best of 3: 62.1 ms per loop


In [5]:
%timeit norm_square_generator_comprehension(vector)

10 loops, best of 3: 63.8 ms per loop


In [6]:
vector = array('l', range(1000000))

In [7]:
%timeit norm_square_array(vector)

10 loops, best of 3: 61.3 ms per loop


In [8]:
vector = numpy.arange(1000000)

In [9]:
%timeit norm_square_numpy(vector)

100 loops, best of 3: 1.9 ms per loop


In [10]:
%timeit norm_square_numpy_dot(vector)

1000 loops, best of 3: 672 µs per loop


In [ ]:
# Numpy Naive Version
from diffusion_numpy_naive import run_experiment
%lprun -f run_experiment run_experiment(500)

'''
Timer unit: 1e-06 s

Total time: 2.85241 s
File: /home/luno/Workspace/high_performance_python/diffusion_numpy_naive.py
Function: run_experiment at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    16                                           def run_experiment(num_iterations):
    17         1           10     10.0      0.0      grid = zeros(grid_shape)
    18                                           
    19         1            2      2.0      0.0      block_low = int(grid_shape[0] * .4)
    20         1            1      1.0      0.0      block_high = int(grid_shape[0] * .5)
    21         1           50     50.0      0.0      grid[block_low:block_high, block_low:block_high] = 0.005
    22                                           
    23         1            1      1.0      0.0      start = time.time()
    24       501         1623      3.2      0.1      for i in range(num_iterations):
    25       500      2850723   5701.4     99.9          grid = evolve(grid, 0.1)
    26         1            4      4.0      0.0      return time.time() - start
'''

```> perf stat -e cycles,stalled-cycles-frontend,stalled-cycles-backend,instructions,\
cache-references,cache-misses,branches,branch-misses,task-clock,faults,\
minor-faults,cs,migrations -r 3 python diffusion_numpy_naive.py


 Performance counter stats for 'python diffusion_numpy_naive.py' (3 runs):

    11,826,392,686      cycles                    #    3.595 GHz                      ( +-  3.90% )
   <not supported>      stalled-cycles-frontend
   <not supported>      stalled-cycles-backend
    11,929,431,699      instructions              #    1.01  insns per cycle          ( +-  1.53% )
       929,662,837      cache-references          #  282.568 M/sec                    ( +-  1.24% )
       389,113,368      cache-misses              #   41.855 % of all cache refs      ( +-  3.40% )
     2,449,998,393      branches                  #  744.670 M/sec                    ( +-  1.06% )
         3,018,605      branch-misses             #    0.12% of all branches          ( +- 11.52% )
       3290.047586      task-clock (msec)         #    1.055 CPUs utilized            ( +-  4.18% )
           750,879      faults                    #    0.228 M/sec                    ( +-  0.00% )
           750,863      minor-faults              #    0.228 M/sec                    ( +-  0.00% )
            64,034      cs                        #    0.019 M/sec                    ( +- 99.70% )
                 8      migrations                #    0.003 K/sec                    ( +- 42.33% )

       3.119952520 seconds time elapsed                                          ( +-  5.83% )```

In [11]:
# In-place operations reducing memory allocations
import numpy as np
array1 = np.random.random((10,10))
array2 = np.random.random((10, 10))
id(array1)

140022980790512

In [12]:
array1 += array2
id(array1)

140022980790512

In [13]:
array1 = array1 + array2
id(array1)

140022980789152

In [15]:
%timeit array1, array2 = np.random.random((10,10)), np.random.random((10,10))

The slowest run took 15.87 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.6 µs per loop


In [16]:
%timeit array1, array2 = np.random.random((10,10)), np.random.random((10,10))

The slowest run took 29.73 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.62 µs per loop


In [ ]:
# Numpy Reducing Memory
from diffusion_numpy_memory import run_experiment
%lprun -f run_experiment run_experiment(500)

'''
Total time: 1.77353 s
File: /home/luno/Workspace/high_performance_python/diffusion_numpy_memory.py
Function: run_experiment at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    22                                           def run_experiment(num_iterations):
    23         1           69     69.0      0.0      scratch = np.zeros(grid_shape)
    24         1          140    140.0      0.0      grid = np.zeros(grid_shape)
    25                                           
    26         1           11     11.0      0.0      block_low = int(grid_shape[0] * .4)
    27         1            3      3.0      0.0      block_high = int(grid_shape[0] * .5)
    28         1          194    194.0      0.0      grid[block_low:block_high, block_low:block_high] = 0.005
    29                                           
    30         1            6      6.0      0.0      start = time.time()
    31       501          375      0.7      0.0      for i in range(num_iterations):
    32       500      1771810   3543.6     99.9          evolve(grid, 0.1, scratch)
    33       500          914      1.8      0.1          grid, scratch = scratch, grid
    34         1            3      3.0      0.0      return time.time() - start
'''

In [ ]:
# Numpy Reducing Memory - More performant, but much less readable
from diffusion_numpy_memory2 import run_experiment
%lprun -f run_experiment run_experiment(500)

'''
Total time: 0.870182 s
File: /home/luno/Workspace/high_performance_python/diffusion_numpy_memory2.py
Function: run_experiment at line 37

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    37                                           def run_experiment(num_iterations):
    38         1           96     96.0      0.0      scratch = zeros(grid_shape)
    39         1          166    166.0      0.0      grid = zeros(grid_shape)
    40                                           
    41         1           10     10.0      0.0      block_low = int(grid_shape[0] * .4)
    42         1            4      4.0      0.0      block_high = int(grid_shape[0] * .5)
    43         1          232    232.0      0.0      grid[block_low:block_high, block_low:block_high] = 0.005
    44                                           
    45         1            9      9.0      0.0      start = time.time()
    46       501          335      0.7      0.0      for i in range(num_iterations):
    47       500       868637   1737.3     99.8          evolve(grid, 0.1, scratch)
    48       500          689      1.4      0.1          grid, scratch = scratch, grid
    49         1            4      4.0      0.0      return time.time() - start
'''

`numexpr`
- take an entire vector expression and compile it to optimized code (less cache misses and temporary space used)
- utilize multiple CPU cores and specialized instructions for Intel chips
- rewrite the expressions as strings with referecnes to local variables.
- Use the various CPU caches that have the correct data in order to minimize cache misses.

In [ ]:
# The number of grid elemetns we can store in total is 20,480 KB / 64 bit = 2,560,000
# for 20,480 KB cache (Intel Xeon ES-2680)
# Since we have two grids, this number is split between two objects ( 2,560,000 / 2 = 1,280,000)
# All in all, array of size 1,131 x 1,131 would fill up the cache
#   sqrt(20480KB / 64 bit /2 ) = 1131


In [ ]:
from numexpr import evaluate
def evolve(grid, dt, next_grid, D=1):
    laplacian(grid, next_grid)
    evaluate("next_grid*D*dt+grid", out=next_grid)

```
> perf stat -e cycles,stalled-cycles-frontend,stalled-cycles-backend,instructions,\
cache-references,cache-misses,branches,branch-misses,task-clock,faults,\
minor-faults,cs,migrations -r 3 python diffusion_numpy_memory2_numexpr.py

 Performance counter stats for 'python diffusion_numpy_memory2_numexpr.py' (3 runs):

     5,119,811,553      cycles                    #    3.464 GHz                      ( +-  4.98% )
   <not supported>      stalled-cycles-frontend
   <not supported>      stalled-cycles-backend
     6,102,912,408      instructions              #    1.19  insns per cycle          ( +-  4.33% )
       603,142,928      cache-references          #  408.125 M/sec                    ( +-  0.45% )
        57,237,731      cache-misses              #    9.490 % of all cache refs      ( +-  4.93% )
       677,596,702      branches                  #  458.505 M/sec                    ( +-  7.76% )
         4,615,015      branch-misses             #    0.68% of all branches          ( +-  1.00% )
       1477.838589      task-clock (msec)         #    1.308 CPUs utilized            ( +-  6.90% )
             9,216      faults                    #    0.006 M/sec                    ( +-  0.11% )
             9,200      minor-faults              #    0.006 M/sec                    ( +-  0.06% )
             4,426      cs                        #    0.003 M/sec                    ( +-  2.77% )
             1,687      migrations                #    0.001 M/sec                    ( +-  0.96% )

       1.129800061 seconds time elapsed                                          ( +-  7.59% )

```

In [ ]:
# A little bit diferent result with textbook, scipy's laplacin code shows 
# higher insns per cycle count and the higher number of `branches` 

Less `page-faults`, but more `
```
perf stat -e cycles,stalled-cycles-frontend,stalled-cycles-backend,instructions,\
cache-references,cache-misses,branches,branch-misses,task-clock,faults,\
minor-faults,cs,migrations -r 3 python diffusion_scipy.py

 Performance counter stats for 'python diffusion_scipy.py' (3 runs):

     8,387,889,741      cycles                    #    3.594 GHz                      ( +-  0.81% )
   <not supported>      stalled-cycles-frontend
   <not supported>      stalled-cycles-backend
    11,150,656,157      instructions              #    1.33  insns per cycle          ( +-  0.14% )
     1,055,765,516      cache-references          #  452.324 M/sec                    ( +-  0.24% )
       134,965,757      cache-misses              #   12.784 % of all cache refs      ( +-  1.19% )
     1,871,308,726      branches                  #  801.729 M/sec                    ( +-  0.16% )
         3,412,571      branch-misses             #    0.18% of all branches          ( +-  0.18% )
       2334.091526      task-clock (msec)         #    1.083 CPUs utilized            ( +-  0.06% )
             7,261      faults                    #    0.003 M/sec                    ( +-  0.18% )
             7,250      minor-faults              #    0.003 M/sec                    ( +-  0.02% )
               416      cs                        #    0.178 K/sec                    ( +- 14.99% )
                 5      migrations                #    0.002 K/sec                    ( +- 25.75% )

       2.154428789 seconds time elapsed                                          ( +-  0.40% )

```

#### Wrap Up
- numpy + memory + laplacian + numexpr (best)
- numpy + memory + laplacian
- numpy + memory
- numpy + memory + scipy
- numpy
- Python + memory
- Python (worst)

#### Additional Links
- [A guide to analyzing Python performance](https://www.huyng.com/posts/python-performance-analysis)
- [Timing and Profiling in IPython](http://pynash.org/2013/03/06/timing-and-profiling/)
- [profile, cProfile, and pstats – Performance analysis of Python programs.](https://pymotw.com/2/profile/)